In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import logging
import os
from pprint import pprint

# Configure logging
logging.basicConfig(level=logging.INFO, filename='scraper.log',
                    format='%(asctime)s:%(levelname)s:%(message)s')

def create_driver():
    """
    Creates and returns a Selenium WebDriver instance using webdriver-manager.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run headlessly (no GUI)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")  # Ensure full page is loaded
    
    # Initialize the ChromeDriver using webdriver-manager
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def scrape_amazon_product(url):
    """
    Scrapes a single Amazon product page.
    Returns a dictionary with product details.
    """
    driver = create_driver()
    product_details = {}

    try:
        logging.info(f"Navigating to Amazon URL: {url}")
        driver.get(url)
        
        wait = WebDriverWait(driver, 10)  # Increased wait time
        
        # Wait for the product title to be present
        name_tag = wait.until(EC.presence_of_element_located((By.ID, 'productTitle')))
        product_details['name'] = name_tag.text.strip()
        logging.info(f"Product Name: {product_details['name']}")

        # Extract price using a more precise CSS selector
        try:
            price_tag = driver.find_element(By.CSS_SELECTOR, 'span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay')
            product_details['price'] = price_tag.text.strip()
            logging.info(f"Price: ₹{product_details['price']}")
        except:
            try:
                price_tag = driver.find_element(By.ID, 'priceblock_dealprice')
                product_details['price'] = price_tag.text.strip()
                logging.info(f"Price: ₹{product_details['price']}")
            except:
                product_details['price'] = 'N/A'
                logging.warning("Price element not found.")
        
        # Extract rating
        try:
            rating_tag = driver.find_element(By.ID, 'averageCustomerReviews')
            product_details['rating'] = rating_tag.text.strip()
            logging.info(f"Rating: {product_details['rating']}")
        except:
            product_details['rating'] = 'N/A'
            logging.warning("Rating element not found.")

        # Extract availability
        try:
            availability_tag = driver.find_element(By.ID, 'availability')
            product_details['availability'] = availability_tag.text.strip()
            logging.info(f"Availability: {product_details['availability']}")
        except:
            product_details['availability'] = 'N/A'
            logging.warning("Availability element not found.")

    except Exception as err:
        logging.error(f"An error occurred while scraping Amazon: {err}")
        # Capture screenshot for debugging
        screenshot_path = os.path.join(os.getcwd(), "amazon_error_screenshot.png")
        driver.save_screenshot(screenshot_path)
        logging.info(f"Screenshot saved to {screenshot_path}")
    finally:
        driver.quit()

    return product_details


def scrape_flipkart_product(url):
    """
    Scrapes a single Flipkart product page.
    Returns a dictionary with product details.
    """
    driver = create_driver()
    product_details = {}

    try:
        logging.info(f"Navigating to Flipkart URL: {url}")
        driver.get(url)
        
        wait = WebDriverWait(driver, 10)  # Increased wait time
        
        # Handle potential login pop-up
        try:
            logging.info("Checking for login pop-up...")
            close_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'✕')]")))
            close_button.click()
            logging.info("Login pop-up closed.")
        except:
            logging.info("No login pop-up detected.")
        
        # Wait for the product name to be present
        name_tag = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.VU-ZEz')))
        product_details['name'] = name_tag.text.strip()
        logging.info(f"Product Name: {product_details['name']}")

        # Extract price using CSS_SELECTOR
        try:
            price_tag = driver.find_element(By.CSS_SELECTOR, 'div.Nx9bqj.CxhGGd')
            product_details['price'] = price_tag.text.strip()
            logging.info(f"Price: {product_details['price']}")
        except:
            product_details['price'] = 'N/A'
            logging.warning("Price element not found.")

        # Extract rating using CSS_SELECTOR
        try:
            rating_tag = driver.find_element(By.CSS_SELECTOR, 'div._5OesEi.HDvrBb')
            product_details['rating'] = rating_tag.text.strip()
            logging.info(f"Rating: {product_details['rating']}")
        except:
            product_details['rating'] = 'N/A'
            logging.warning("Rating element not found.")

        # Extract availability
        try:
            # Flipkart does not have an element with ID 'availability'
            # Instead, check for 'In stock' or similar text
            availability_tag = driver.find_element(By.XPATH, "//div[contains(text(),'Availability')]/following-sibling::div")
            product_details['availability'] = availability_tag.text.strip()
            logging.info(f"Availability: {product_details['availability']}")
        except:
            # Alternative approach: Check for 'Add to cart' button
            try:
                add_to_cart = driver.find_element(By.XPATH, "//button[contains(text(),'Add to cart')]")
                product_details['availability'] = 'In stock'
                logging.info("Availability inferred from 'Add to cart' button.")
            except:
                product_details['availability'] = 'N/A'
                logging.warning("Availability element not found.")

    except Exception as err:
        logging.error(f"An error occurred while scraping Flipkart: {err}")
        # Capture screenshot for debugging
        screenshot_path = os.path.join(os.getcwd(), "flipkart_error_screenshot.png")
        driver.save_screenshot(screenshot_path)
        logging.info(f"Screenshot saved to {screenshot_path}")
    finally:
        driver.quit()

    return product_details

# ----------------- Added JioMart Scraper Below ----------------- #

def scrape_jiomart_product(url):
    """
    Scrapes a single JioMart product page.
    Returns a dictionary with product details.
    """
    driver = create_driver()
    product_details = {}

    try:
        logging.info(f"Navigating to JioMart URL: {url}")
        driver.get(url)
        
        wait = WebDriverWait(driver, 10)  # Increased wait time for dynamic content

        # Wait for the product name to be present
        name_tag = wait.until(EC.presence_of_element_located((By.ID, 'pdp_product_name')))
        product_details['name'] = name_tag.text.strip()
        logging.info(f"Product Name: {product_details['name']}")

        # Extract price
        try:
            # Updated selector based on provided HTML
            price_tag = driver.find_element(By.CSS_SELECTOR, 'span.jm-heading-xs.jm-ml-xxs')
            product_details['price'] = price_tag.text.strip()
            logging.info(f"Price: {product_details['price']}")
        except Exception as e:
            product_details['price'] = 'N/A'
            logging.warning("Price element not found.")

        # Extract rating
        try:
            # Locate the container with filled stars
            rating_filled_container = driver.find_element(By.CSS_SELECTOR, 'div.jm-rating-filled')
            filled_stars = rating_filled_container.find_elements(By.TAG_NAME, 'div')

            # Initialize rating
            rating = 0.0

            for star in filled_stars:
                star_id = star.get_attribute('id')
                if star_id:
                    try:
                        star_value = float(star_id)
                        if star_value > rating:
                            rating = star_value
                    except ValueError:
                        continue  # Skip if id is not a float

            if rating > 0:
                product_details['rating'] = str(rating)
                logging.info(f"Rating: {product_details['rating']}")
            else:
                product_details['rating'] = 'N/A'
                logging.warning("Rating value could not be determined.")
        except Exception as e:
            product_details['rating'] = 'N/A'
            logging.warning("Rating element not found or could not be parsed.")

        # Extract availability
        try:
            # Attempt to find stock status
            availability_tag = driver.find_element(By.CSS_SELECTOR, 'span.stock-status')
            product_details['availability'] = availability_tag.text.strip()
            logging.info(f"Availability: {product_details['availability']}")
        except:
            # Alternative approach: Check for 'Add to Cart' button
            try:
                add_to_cart = driver.find_element(By.XPATH, "//button[contains(text(),'Add to Cart')]")
                product_details['availability'] = 'In stock'
                logging.info("Availability inferred from 'Add to Cart' button.")
            except:
                product_details['availability'] = 'N/A'
                logging.warning("Availability element not found.")

    except Exception as err:
        logging.error(f"An error occurred while scraping JioMart: {err}")
        # Capture screenshot for debugging
        screenshot_path = os.path.join(os.getcwd(), "jiomart_error_screenshot.png")
        driver.save_screenshot(screenshot_path)
        logging.info(f"Screenshot saved to {screenshot_path}")
    finally:
        driver.quit()

    return product_details


# -------------------------------------------------------------------- #

def get_all_product_details(amazon_url=None, flipkart_url=None, jiomart_url=None):
    """
    Aggregates product details from specified e-commerce platforms.
    Accepts URLs for Amazon, Flipkart, and JioMart product pages.
    """
    combined_data = {}
    
    if amazon_url:
        amazon_data = scrape_amazon_product(amazon_url)
        combined_data['Amazon'] = amazon_data
    
    if flipkart_url:
        flipkart_data = scrape_flipkart_product(flipkart_url)
        combined_data['Flipkart'] = flipkart_data
    
    if jiomart_url:
        jiomart_data = scrape_jiomart_product(jiomart_url)
        combined_data['JioMart'] = jiomart_data
    
    return combined_data

if __name__ == "__main__":
    # Example product URLs
    amazon_product_url = "https://www.amazon.in/CMF-NOTHING-Phone-128-Black/dp/B0D935PW3P/ref=sr_1_1?crid=2PRDA56B939J7&dib=eyJ2IjoiMSJ9.KrVtzXq8mQhRcpLxA2_vnRYAnkji9EdvgXfRn7E6b7K_kreEyPBR1AarFeppRoVvHs2MdG-FajNtlnaB2V1iyiXzl4wbhI1lp6gzav0c1nHagtus1-xDiSFFhXnbxrim1yyjZfTrSW_VsBYZpGeifEDSlyUoXhpxwD510Kz0JSBgCDNHqdWh1VOazSzodWaJA7Me_r4g3AOS-FWxIxisCL84Du2qJskAlmtDafQbJAc.eeNgiPO-UXmfXQGZCQ1XqxpubS9cI-aeke2tv2mwkWo&dib_tag=se&keywords=cmp+nothing+mobile&nsdOptOutParam=true&qid=1727800194&sprefix=cmp+nothing+%2Caps%2C256&sr=8-1"  # Replace with your desired Amazon product URL
    flipkart_product_url = "https://www.flipkart.com/cmf-nothing-phone-1-black-128-gb/p/itmeef68c7ce70bf?pid=MOBHYBQTGGEGGA2B&lid=LSTMOBHYBQTGGEGGA2BCRTZZY&param=79233&ctx=eyJjYXJkQ29udGV4dCI6eyJhdHRyaWJ1dGVzIjp7InZhbHVlQ2FsbG91dCI6eyJtdWx0aVZhbHVlZEF0dHJpYnV0ZSI6eyJrZXkiOiJ2YWx1ZUNhbGxvdXQiLCJpbmZlcmVuY2VUeXBlIjoiVkFMVUVfQ0FMTE9VVCIsInZhbHVlcyI6WyJEaXNwbGF5IDE3LjAyIGNtIl0sInZhbHVlVHlwZSI6Ik1VTFRJX1ZBTFVFRCJ9fSwidGl0bGUiOnsibXVsdGlWYWx1ZWRBdHRyaWJ1dGUiOnsia2V5IjoidGl0bGUiLCJpbmZlcmVuY2VUeXBlIjoiVElUTEUiLCJ2YWx1ZXMiOlsiQ01GIFBob25lIDEiXSwidmFsdWVUeXBlIjoiTVVMVElfVkFMVUVEIn19LCJoZXJvUGlkIjp7InNpbmdsZVZhbHVlQXR0cmlidXRlIjp7ImtleSI6Imhlcm9QaWQiLCJpbmZlcmVuY2VUeXBlIjoiUElEIiwidmFsdWUiOiJNT0JIWUJRVEdHRUdHQTJCIiwidmFsdWVUeXBlIjoiU0lOR0xFX1ZBTFVFRCJ9fX19fQ%3D%3D"  # Replace with your desired Flipkart product URL
    jiomart_product_url = "https://www.jiomart.com/p/electronics/nothing-cmf-phone-1-6gb-ram-128gb-rom-black-smartphone/609538384"  # Replace with your desired JioMart product URL

    product_details = get_all_product_details(
        amazon_url=amazon_product_url,
        flipkart_url=flipkart_product_url,
        jiomart_url=jiomart_product_url  # Added JioMart URL
    )
    
    pprint(product_details)


{'Amazon': {'availability': 'Only 1 left in stock.',
            'name': 'CMF BY NOTHING Phone 1 5G (128 GB) (6 GB RAM) (Black)',
            'price': '₹15,770',
            'rating': '4.2\n73 ratings'},
 'Flipkart': {'availability': 'In stock',
              'name': 'CMF by Nothing Phone 1 (Black, 128 GB)  (6 GB RAM)',
              'price': '₹14,999',
              'rating': '4.427,915 Ratings & 2,024 Reviews'},
 'JioMart': {'availability': 'In stock',
             'name': 'Nothing CMF Phone 1, 6GB RAM, 128GB ROM, Black, '
                     'Smartphone',
             'price': '₹15,999.00',
             'rating': '3.6'}}
